
---

# **Chapter 28: Unit Testing**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Test pure functions and business logic with complete isolation
- Implement repository pattern testing with local and remote data sources
- Test state management solutions (BLoC, Provider, Riverpod) using specialized testing utilities
- Handle asynchronous operations and Futures in unit tests effectively
- Test Streams including broadcast streams, transformers, and error scenarios
- Analyze and interpret code coverage metrics to identify testing gaps
- Implement dependency injection strategies for testable architectures

---

## **Prerequisites**

- Completed Chapter 27: Testing Fundamentals (AAA pattern, mocking basics)
- Understanding of Repository Pattern and Clean Architecture
- Familiarity with state management concepts (BLoC, Provider, or Riverpod)
- Knowledge of Streams and async/await patterns from Chapter 6
- Flutter SDK with `flutter_test`, `mocktail`, and `bloc_test` (for BLoC sections) packages

---

## **28.1 Testing Pure Functions and Business Logic**

Pure functions are the easiest and most important code to test. Given the same inputs, they always produce the same outputs without side effects.

### **Testing Mathematical Operations**

```dart
// File: lib/domain/utils/calculator.dart
class Calculator {
  // Pure function: output depends only on inputs, no side effects
  double add(double a, double b) => a + b;
  
  double multiply(double a, double b) => a * b;
  
  double calculateCompoundInterest(
    double principal,
    double rate,
    int time,
    int frequency,
  ) {
    // Formula: A = P(1 + r/n)^(nt)
    final amount = principal * 
        pow((1 + rate / frequency), (frequency * time));
    return amount - principal; // Return only interest
  }
  
  double pow(double base, int exponent) {
    // Custom power function for demonstration
    double result = 1;
    for (var i = 0; i < exponent; i++) {
      result *= base;
    }
    return result;
  }
}

// File: test/domain/utils/calculator_test.dart
import 'package:flutter_test/flutter_test.dart';
import 'package:my_app/domain/utils/calculator.dart';

void main() {
  // Group related functionality
  group('Calculator', () {
    // Declare system under test (SUT)
    late Calculator calculator;
    
    // Fresh instance before each test ensures isolation
    setUp(() {
      calculator = Calculator();
    });
    
    // Test simple addition with integers
    test('add should return sum of two positive integers', () {
      // Arrange: Define inputs and expected output
      const a = 5.0;
      const b = 3.0;
      const expected = 8.0;
      
      // Act: Execute the function
      final result = calculator.add(a, b);
      
      // Assert: Verify exact equality for doubles
      expect(result, equals(expected));
    });
    
    // Test edge case: negative numbers
    test('add should handle negative numbers correctly', () {
      expect(calculator.add(-5, 3), equals(-2));
      expect(calculator.add(-5, -3), equals(-8));
      expect(calculator.add(0, 0), equals(0));
    });
    
    // Test floating point precision
    test('add should handle decimal precision', () {
      // Use closeTo matcher for floating point comparisons
      // to avoid precision errors (0.1 + 0.2 != 0.3 in binary)
      final result = calculator.add(0.1, 0.2);
      expect(result, closeTo(0.3, 0.0001));
      // closeTo checks if value is within 0.0001 of 0.3
    });
    
    // Parametrized test using a data table
    test('multiply should return correct products', () {
      // Define test cases as tuples (input1, input2, expected)
      final testCases = [
        [2.0, 3.0, 6.0],
        [0.0, 5.0, 0.0],
        [-2.0, 3.0, -6.0],
        [-2.0, -3.0, 6.0],
      ];
      
      // Iterate through all test cases
      for (final testCase in testCases) {
        final a = testCase[0];
        final b = testCase[1];
        final expected = testCase[2];
        
        final result = calculator.multiply(a, b);
        
        expect(
          result, 
          equals(expected),
          reason: 'Failed for inputs: $a * $b',
        );
        // reason helps identify which specific case failed
      }
    });
    
    // Complex business logic test
    group('calculateCompoundInterest', () {
      test('should calculate correct interest for annual compounding', () {
        // Arrange: $1000 at 5% for 10 years, compounded annually
        const principal = 1000.0;
        const rate = 0.05;
        const time = 10;
        const frequency = 1; // Annual
        
        // Act
        final interest = calculator.calculateCompoundInterest(
          principal,
          rate,
          time,
          frequency,
        );
        
        // Assert: Expected ~$628.89 interest
        expect(interest, closeTo(628.89, 0.01));
      });
      
      test('should calculate correct interest for monthly compounding', () {
        // Monthly compounding yields slightly more interest
        const principal = 1000.0;
        const rate = 0.05;
        const time = 10;
        const frequency = 12; // Monthly
        
        final interest = calculator.calculateCompoundInterest(
          principal,
          rate,
          time,
          frequency,
        );
        
        // Monthly should yield ~$647.01 (more than annual)
        expect(interest, closeTo(647.01, 0.01));
        expect(interest, greaterThan(628.89)); // More than annual
      });
    });
  });
}
```

**Explanation:**

- **Pure functions**: The `Calculator` class contains pure functions—no external dependencies, no mutable state, no I/O operations. They are deterministic and testable.
- **`closeTo` matcher**: Essential for floating-point comparisons. Due to binary representation, `0.1 + 0.2` equals `0.30000000000000004`, not `0.3`. `closeTo` verifies the value is within an acceptable error margin (delta).
- **Parameterized testing**: Using a list of test cases (`testCases`) allows testing multiple inputs with the same assertion logic. This reduces code duplication while maintaining clarity.
- **`reason` parameter**: When iterating through multiple test cases, the `reason` parameter in `expect()` identifies exactly which iteration failed, making debugging easier.
- **Mathematical accuracy**: Tests verify not just that calculations work, but that they match real-world financial formulas with precision requirements.

### **Testing Validation Logic**

```dart
// File: lib/domain/validators/password_validator.dart

// Immutable value object representing validation result
class ValidationResult {
  final bool isValid;
  final String? errorMessage;
  final List<ValidationError> errors;
  
  const ValidationResult({
    required this.isValid,
    this.errorMessage,
    this.errors = const [],
  });
  
  // Factory constructor for valid result
  factory ValidationResult.valid() => 
      const ValidationResult(isValid: true);
  
  // Factory constructor for invalid result
  factory ValidationResult.invalid(String message, List<ValidationError> errors) => 
      ValidationResult(isValid: false, errorMessage: message, errors: errors);
}

enum ValidationError {
  tooShort,
  noUppercase,
  noLowercase,
  noDigit,
  noSpecialCharacter,
}

class PasswordValidator {
  // Business rules as constants for easy modification
  static const int minLength = 8;
  static const int maxLength = 128;
  
  ValidationResult validate(String password) {
    // Collect all errors to provide comprehensive feedback
    final errors = <ValidationError>[];
    
    // Rule 1: Length check
    if (password.length < minLength) {
      errors.add(ValidationError.tooShort);
    }
    if (password.length > maxLength) {
      return ValidationResult.invalid(
        'Password must not exceed $maxLength characters',
        [ValidationError.tooShort], // Using tooShort as proxy for length issues
      );
    }
    
    // Rule 2: Character variety checks using RegExp
    if (!RegExp(r'[A-Z]').hasMatch(password)) {
      errors.add(ValidationError.noUppercase);
    }
    if (!RegExp(r'[a-z]').hasMatch(password)) {
      errors.add(ValidationError.noLowercase);
    }
    if (!RegExp(r'[0-9]').hasMatch(password)) {
      errors.add(ValidationError.noDigit);
    }
    if (!RegExp(r'[!@#$%^&*(),.?":{}|<>]').hasMatch(password)) {
      errors.add(ValidationError.noSpecialCharacter);
    }
    
    // Return appropriate result based on collected errors
    if (errors.isEmpty) {
      return ValidationResult.valid();
    } else {
      return ValidationResult.invalid(
        _buildErrorMessage(errors),
        errors,
      );
    }
  }
  
  String _buildErrorMessage(List<ValidationError> errors) {
    final buffer = StringBuffer('Password must contain: ');
    final messages = errors.map((e) {
      switch (e) {
        case ValidationError.tooShort:
          return 'at least $minLength characters';
        case ValidationError.noUppercase:
          return 'uppercase letter';
        case ValidationError.noLowercase:
          return 'lowercase letter';
        case ValidationError.noDigit:
          return 'digit';
        case ValidationError.noSpecialCharacter:
          return 'special character';
      }
    }).join(', ');
    
    buffer.write(messages);
    return buffer.toString();
  }
}

// File: test/domain/validators/password_validator_test.dart
import 'package:flutter_test/flutter_test.dart';
import 'package:my_app/domain/validators/password_validator.dart';

void main() {
  group('PasswordValidator', () {
    late PasswordValidator validator;
    
    setUp(() {
      validator = PasswordValidator();
    });
    
    test('should return valid for strong password', () {
      // Arrange: Password meeting all criteria
      const strongPassword = 'StrongP@ssw0rd';
      
      // Act
      final result = validator.validate(strongPassword);
      
      // Assert: Check all properties of the result
      expect(result.isValid, isTrue);
      expect(result.errorMessage, isNull);
      expect(result.errors, isEmpty);
    });
    
    test('should detect missing uppercase', () {
      // Arrange: lowercase, digits, special chars, but no uppercase
      const password = 'weakp@ssw0rd';
      
      // Act
      final result = validator.validate(password);
      
      // Assert
      expect(result.isValid, isFalse);
      expect(result.errors, contains(ValidationError.noUppercase));
      expect(result.errors, hasLength(1)); // Only one error expected
    });
    
    test('should detect multiple validation failures', () {
      // Arrange: Only 5 chars, no uppercase, no special char
      const password = 'weak1';
      
      // Act
      final result = validator.validate(password);
      
      // Assert: Check for multiple specific errors
      expect(result.errors, contains(ValidationError.tooShort));
      expect(result.errors, contains(ValidationError.noUppercase));
      expect(result.errors, contains(ValidationError.noSpecialCharacter));
      expect(result.errors, hasLength(3));
      
      // Verify error message construction
      expect(
        result.errorMessage, 
        contains('at least 8 characters'),
      );
      expect(result.errorMessage, contains('uppercase letter'));
      expect(result.errorMessage, contains('special character'));
    });
    
    test('should detect missing digits', () {
      final result = validator.validate('NoDigitsHere!');
      expect(result.errors, contains(ValidationError.noDigit));
    });
    
    test('should detect missing special characters', () {
      final result = validator.validate('NoSpecialChars123');
      expect(result.errors, contains(ValidationError.noSpecialCharacter));
    });
    
    test('should reject passwords exceeding max length', () {
      // Arrange: Create string longer than 128 chars
      final tooLong = 'A' * 129 + '!1';
      
      final result = validator.validate(tooLong);
      
      expect(result.isValid, isFalse);
      expect(result.errorMessage, contains('128'));
    });
    
    // Boundary value analysis
    test('should accept password at minimum length', () {
      // Exactly 8 chars: Aa1!aaaa (8 characters)
      final result = validator.validate('Aa1!aaaa');
      expect(result.isValid, isTrue);
    });
    
    test('should reject password at minimum length minus one', () {
      // 7 chars: Aa1!aaa (7 characters)
      final result = validator.validate('Aa1!aaa');
      expect(result.isValid, isFalse);
      expect(result.errors, contains(ValidationError.tooShort));
    });
  });
}
```

**Explanation:**

- **Value Objects**: `ValidationResult` is an immutable object that encapsulates both the validity state and detailed error information. This allows tests to verify specific failure modes, not just pass/fail.
- **Enum-based errors**: Using `ValidationError` enum instead of just strings allows tests to verify specific business rules were violated (e.g., `contains(ValidationError.noUppercase)`).
- **Boundary Value Analysis**: Testing at the edges of valid input (exactly 8 characters vs. 7 characters) catches off-by-one errors common in validation logic.
- **Comprehensive error collection**: The validator collects all errors rather than failing fast. This allows users to see all requirements they missed, and allows tests to verify multiple constraints simultaneously.
- **`hasLength` matcher**: Verifies the exact number of errors returned, ensuring the validator doesn't miss constraints or report duplicates.

---

## **28.2 Testing Repository Pattern Implementations**

The Repository Pattern abstracts data access. Testing repositories involves verifying they correctly coordinate between local and remote data sources.

### **Testing Repository with Local and Remote Sources**

```dart
// File: lib/domain/repositories/user_repository.dart
abstract class UserRepository {
  Future<User> getUser(String id);
  Future<void> saveUser(User user);
  Future<List<User>> getAllUsers();
}

// File: lib/data/models/user_model.dart
class UserModel extends User {
  final String createdAt;
  final String updatedAt;
  
  UserModel({
    required String id,
    required String name,
    required String email,
    required this.createdAt,
    required this.updatedAt,
  }) : super(id: id, name: name, email: email);
  
  factory UserModel.fromJson(Map<String, dynamic> json) {
    return UserModel(
      id: json['id'],
      name: json['name'],
      email: json['email'],
      createdAt: json['created_at'],
      updatedAt: json['updated_at'],
    );
  }
  
  Map<String, dynamic> toJson() => {
    'id': id,
    'name': name,
    'email': email,
    'created_at': createdAt,
    'updated_at': updatedAt,
  };
  
  // Convert domain entity to model
  factory UserModel.fromEntity(User user) => UserModel(
    id: user.id,
    name: user.name,
    email: user.email,
    createdAt: DateTime.now().toIso8601String(),
    updatedAt: DateTime.now().toIso8601String(),
  );
}

// File: lib/data/repositories/user_repository_impl.dart
class UserRepositoryImpl implements UserRepository {
  final RemoteDataSource remoteDataSource;
  final LocalDataSource localDataSource;
  final NetworkInfo networkInfo;
  
  UserRepositoryImpl({
    required this.remoteDataSource,
    required this.localDataSource,
    required this.networkInfo,
  });
  
  @override
  Future<User> getUser(String id) async {
    // Check network connectivity first
    if (await networkInfo.isConnected) {
      try {
        // Fetch from remote
        final remoteUser = await remoteDataSource.getUser(id);
        
        // Cache locally for offline access
        await localDataSource.cacheUser(remoteUser);
        
        return remoteUser;
      } on ServerException {
        // If server fails, try local cache
        return await _getUserFromLocal(id);
      }
    } else {
      // Offline: get from local
      return await _getUserFromLocal(id);
    }
  }
  
  Future<User> _getUserFromLocal(String id) async {
    final localUser = await localDataSource.getLastUser(id);
    if (localUser != null) {
      return localUser;
    } else {
      throw CacheException('No cached user found');
    }
  }
  
  @override
  Future<void> saveUser(User user) async {
    final model = UserModel.fromEntity(user);
    
    if (await networkInfo.isConnected) {
      await remoteDataSource.saveUser(model);
    }
    
    // Always save locally
    await localDataSource.cacheUser(model);
  }
  
  @override
  Future<List<User>> getAllUsers() async {
    if (await networkInfo.isConnected) {
      final remoteUsers = await remoteDataSource.getAllUsers();
      await localDataSource.cacheUsers(remoteUsers);
      return remoteUsers;
    } else {
      return await localDataSource.getCachedUsers();
    }
  }
}

// File: test/data/repositories/user_repository_impl_test.dart
import 'package:flutter_test/flutter_test.dart';
import 'package:mocktail/mocktail.dart';
import 'package:my_app/data/models/user_model.dart';
import 'package:my_app/data/repositories/user_repository_impl.dart';
import 'package:my_app/domain/entities/user.dart';

// Mock dependencies
class MockRemoteDataSource extends Mock implements RemoteDataSource {}
class MockLocalDataSource extends Mock implements LocalDataSource {}
class MockNetworkInfo extends Mock implements NetworkInfo {}

void main() {
  // Declare SUT and mocks
  late UserRepositoryImpl repository;
  late MockRemoteDataSource mockRemote;
  late MockLocalDataSource mockLocal;
  late MockNetworkInfo mockNetwork;
  
  // Test data
  final tId = '123';
  final tUserModel = UserModel(
    id: '123',
    name: 'Test User',
    email: 'test@example.com',
    createdAt: '2023-01-01',
    updatedAt: '2023-01-01',
  );
  
  setUp(() {
    // Initialize mocks
    mockRemote = MockRemoteDataSource();
    mockLocal = MockLocalDataSource();
    mockNetwork = MockNetworkInfo();
    
    // Initialize SUT with injected mocks
    repository = UserRepositoryImpl(
      remoteDataSource: mockRemote,
      localDataSource: mockLocal,
      networkInfo: mockNetwork,
    );
  });
  
  group('getUser', () {
    // Online scenarios
    group('device is online', () {
      setUp(() {
        // Default setup for online tests
        when(() => mockNetwork.isConnected).thenAnswer((_) async => true);
      });
      
      test('should return remote data when call is successful', () async {
        // Arrange: Setup remote success
        when(() => mockRemote.getUser(any()))
            .thenAnswer((_) async => tUserModel);
        when(() => mockLocal.cacheUser(any()))
            .thenAnswer((_) async => {});
        
        // Act
        final result = await repository.getUser(tId);
        
        // Assert: Verify correct data returned
        expect(result, equals(tUserModel));
        
        // Verify remote was called
        verify(() => mockRemote.getUser(tId)).called(1);
        
        // Verify caching occurred (side effect)
        verify(() => mockLocal.cacheUser(tUserModel)).called(1);
      });
      
      test('should cache data locally when remote fetch succeeds', () async {
        // Arrange
        when(() => mockRemote.getUser(any()))
            .thenAnswer((_) async => tUserModel);
        when(() => mockLocal.cacheUser(any()))
            .thenAnswer((_) async => {});
        
        // Act
        await repository.getUser(tId);
        
        // Assert: Verify caching interaction
        verify(() => mockLocal.cacheUser(tUserModel)).called(1);
      });
      
      test('should return local data when remote fails', () async {
        // Arrange: Remote fails, local succeeds
        when(() => mockRemote.getUser(any()))
            .thenThrow(ServerException());
        when(() => mockLocal.getLastUser(any()))
            .thenAnswer((_) async => tUserModel);
        
        // Act
        final result = await repository.getUser(tId);
        
        // Assert
        expect(result, equals(tUserModel));
        verify(() => mockRemote.getUser(tId)).called(1);
        verify(() => mockLocal.getLastUser(tId)).called(1);
        // Should not try to cache if remote failed
        verifyNever(() => mockLocal.cacheUser(any()));
      });
    });
    
    // Offline scenarios
    group('device is offline', () {
      setUp(() {
        when(() => mockNetwork.isConnected).thenAnswer((_) async => false);
      });
      
      test('should return local data when offline', () async {
        // Arrange
        when(() => mockLocal.getLastUser(any()))
            .thenAnswer((_) async => tUserModel);
        
        // Act
        final result = await repository.getUser(tId);
        
        // Assert
        expect(result, equals(tUserModel));
        // Should not call remote when offline
        verifyNever(() => mockRemote.getUser(any()));
        verify(() => mockLocal.getLastUser(tId)).called(1);
      });
      
      test('should throw CacheException when no local data exists', () async {
        // Arrange: No cached data
        when(() => mockLocal.getLastUser(any()))
            .thenAnswer((_) async => null);
        
        // Act & Assert
        expect(
          () => repository.getUser(tId),
          throwsA(isA<CacheException>()),
        );
      });
    });
  });
  
  group('saveUser', () {
    final tUser = User(
      id: '123',
      name: 'Test User',
      email: 'test@example.com',
    );
    
    test('should save to remote when online', () async {
      // Arrange
      when(() => mockNetwork.isConnected).thenAnswer((_) async => true);
      when(() => mockRemote.saveUser(any()))
          .thenAnswer((_) async => {});
      when(() => mockLocal.cacheUser(any()))
          .thenAnswer((_) async => {});
      
      // Act
      await repository.saveUser(tUser);
      
      // Assert
      verify(() => mockRemote.saveUser(any())).called(1);
      verify(() => mockLocal.cacheUser(any())).called(1);
    });
    
    test('should save to local only when offline', () async {
      // Arrange
      when(() => mockNetwork.isConnected).thenAnswer((_) async => false);
      when(() => mockLocal.cacheUser(any()))
          .thenAnswer((_) async => {});
      
      // Act
      await repository.saveUser(tUser);
      
      // Assert
      verifyNever(() => mockRemote.saveUser(any()));
      verify(() => mockLocal.cacheUser(any())).called(1);
    });
  });
}
```

**Explanation:**

- **Repository Pattern**: The `UserRepositoryImpl` coordinates between `RemoteDataSource` (API) and `LocalDataSource` (Cache/DB). Tests verify this coordination logic without hitting real networks or databases.
- **Group nesting**: Tests are organized by method (`getUser`) and then by condition (`device is online/offline`). This creates readable test reports that mirror the logic branches.
- **Side effect verification**: When the repository fetches from remote, it should cache locally. Tests verify this with `verify(() => mockLocal.cacheUser(...))`.
- **Exception handling**: Tests verify graceful degradation—when remote fails, it should fall back to local. When local is empty offline, it should throw `CacheException`.
- **`verifyNever()`**: Ensures that expensive operations (network calls) don't happen when they shouldn't (e.g., when offline).
- **Argument capture**: `any()` is used when the specific instance doesn't matter (though we verify it was called), while specific values (`tId`) verify correct parameters are passed through.

---

## **28.3 Testing State Management (BLoC, Provider, Riverpod)**

State management testing verifies that business logic components (BLoCs, ViewModels) emit correct states in response to events.

### **Testing BLoC (Business Logic Component)**

```dart
// File: lib/presentation/bloc/auth_bloc.dart
import 'package:flutter_bloc/flutter_bloc.dart';

// Events
abstract class AuthEvent {}

class LoginRequested extends AuthEvent {
  final String email;
  final String password;
  LoginRequested(this.email, this.password);
}

class LogoutRequested extends AuthEvent {}

// States
abstract class AuthState {}

class AuthInitial extends AuthState {}

class AuthLoading extends AuthState {}

class AuthAuthenticated extends AuthState {
  final User user;
  AuthAuthenticated(this.user);
}

class AuthError extends AuthState {
  final String message;
  AuthError(this.message);
}

// BLoC
class AuthBloc extends Bloc<AuthEvent, AuthState> {
  final LoginUseCase loginUseCase;
  final LogoutUseCase logoutUseCase;
  
  AuthBloc({
    required this.loginUseCase,
    required this.logoutUseCase,
  }) : super(AuthInitial()) {
    // Register event handlers
    on<LoginRequested>(_onLoginRequested);
    on<LogoutRequested>(_onLogoutRequested);
  }
  
  Future<void> _onLoginRequested(
    LoginRequested event,
    Emitter<AuthState> emit,
  ) async {
    // Emit loading state immediately
    emit(AuthLoading());
    
    try {
      // Execute use case
      final user = await loginUseCase(
        LoginParams(email: event.email, password: event.password),
      );
      
      // Emit success state
      emit(AuthAuthenticated(user));
    } on InvalidCredentialsException {
      emit(AuthError('Invalid email or password'));
    } on NetworkException {
      emit(AuthError('Network error. Please try again.'));
    } catch (e) {
      emit(AuthError('An unexpected error occurred'));
    }
  }
  
  Future<void> _onLogoutRequested(
    LogoutRequested event,
    Emitter<AuthState> emit,
  ) async {
    await logoutUseCase();
    emit(AuthInitial());
  }
}

// File: test/presentation/bloc/auth_bloc_test.dart
import 'package:bloc_test/bloc_test.dart';
import 'package:flutter_test/flutter_test.dart';
import 'package:mocktail/mocktail.dart';

class MockLoginUseCase extends Mock implements LoginUseCase {}
class MockLogoutUseCase extends Mock implements LogoutUseCase {}

void main() {
  late AuthBloc authBloc;
  late MockLoginUseCase mockLogin;
  late MockLogoutUseCase mockLogout;
  
  // Test data
  final tUser = User(id: '1', email: 'test@example.com', name: 'Test');
  final tEmail = 'test@example.com';
  final tPassword = 'password123';
  
  setUp(() {
    mockLogin = MockLoginUseCase();
    mockLogout = MockLogoutUseCase();
    
    authBloc = AuthBloc(
      loginUseCase: mockLogin,
      logoutUseCase: mockLogout,
    );
  });
  
  // Clean up resources
  tearDown(() {
    authBloc.close();
  });
  
  // Initial state test
  test('initial state should be AuthInitial', () {
    expect(authBloc.state, equals(AuthInitial()));
  });
  
  // blocTest helper from bloc_test package simplifies BLoC testing
  group('LoginRequested', () {
    blocTest<AuthBloc, AuthState>(
      'should emit [AuthLoading, AuthAuthenticated] when login succeeds',
      // build: Create the BLoC instance
      build: () {
        // Setup mock success
        when(() => mockLogin(any()))
            .thenAnswer((_) async => tUser);
        return authBloc;
      },
      // act: Fire events into the BLoC
      act: (bloc) => bloc.add(LoginRequested(tEmail, tPassword)),
      // expect: Define expected state sequence
      expect: () => [
        AuthLoading(),
        AuthAuthenticated(tUser),
      ],
      // verify: Verify interactions with mocks
      verify: (_) {
        verify(() => mockLogin(LoginParams(
          email: tEmail, 
          password: tPassword,
        ))).called(1);
      },
    );
    
    blocTest<AuthBloc, AuthState>(
      'should emit [AuthLoading, AuthError] when credentials invalid',
      build: () {
        when(() => mockLogin(any()))
            .thenThrow(InvalidCredentialsException());
        return authBloc;
      },
      act: (bloc) => bloc.add(LoginRequested(tEmail, tPassword)),
      expect: () => [
        AuthLoading(),
        AuthError('Invalid email or password'),
      ],
    );
    
    blocTest<AuthBloc, AuthState>(
      'should emit [AuthLoading, AuthError] on network failure',
      build: () {
        when(() => mockLogin(any()))
            .thenThrow(NetworkException());
        return authBloc;
      },
      act: (bloc) => bloc.add(LoginRequested(tEmail, tPassword)),
      expect: () => [
        AuthLoading(),
        AuthError('Network error. Please try again.'),
      ],
    );
  });
  
  group('LogoutRequested', () {
    blocTest<AuthBloc, AuthState>(
      'should emit [AuthInitial] when logout requested',
      build: () {
        when(() => mockLogout()).thenAnswer((_) async {});
        return authBloc;
      },
      act: (bloc) => bloc.add(LogoutRequested()),
      expect: () => [AuthInitial()],
      verify: (_) {
        verify(() => mockLogout()).called(1);
      },
    );
  });
}
```

**Explanation:**

- **`bloc_test` package**: Provides the `blocTest` helper which handles the complexity of testing Streams. It collects all states emitted during `act` and compares them to `expect`.
- **State sequence**: BLoCs emit states over time. `expect: () => [State1, State2]` verifies the exact order and values of states.
- **Initial state test**: A simple `test()` (not `blocTest`) verifies the BLoC starts in the correct state before any events.
- **`tearDown()`**: Closes the BLoC after each test to prevent memory leaks and Stream subscription issues.
- **Event-to-State mapping**: Each test verifies a specific event triggers the correct sequence of states. For login: first loading (UI shows spinner), then either success (navigate to home) or error (show message).
- **Exception mapping**: The BLoC catches various exceptions and maps them to user-friendly error states. Tests verify this mapping logic.

### **Testing Provider/Riverpod**

```dart
// File: lib/presentation/providers/counter_provider.dart

// Simple ChangeNotifier for Provider
class CounterNotifier extends ChangeNotifier {
  int _count = 0;
  int get count => _count;
  
  void increment() {
    _count++;
    notifyListeners(); // Triggers UI rebuild
  }
  
  void decrement() {
    _count--;
    notifyListeners();
  }
  
  void reset() {
    _count = 0;
    notifyListeners();
  }
}

// Riverpod StateNotifier
class CounterState extends StateNotifier<int> {
  CounterState() : super(0); // Initial state is 0
  
  void increment() => state++;
  void decrement() => state--;
  void reset() => state = 0;
}

// Riverpod provider definition
final counterProvider = StateNotifierProvider<CounterState, int>((ref) {
  return CounterState();
});

// File: test/presentation/providers/counter_provider_test.dart

// Testing ChangeNotifier (Provider pattern)
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('CounterNotifier (Provider)', () {
    late CounterNotifier notifier;
    
    setUp(() {
      notifier = CounterNotifier();
    });
    
    test('should start at 0', () {
      expect(notifier.count, equals(0));
    });
    
    test('increment should increase count by 1', () {
      // Act
      notifier.increment();
      
      // Assert
      expect(notifier.count, equals(1));
    });
    
    test('decrement should decrease count by 1', () {
      // Arrange: Start at 1
      notifier.increment();
      expect(notifier.count, equals(1));
      
      // Act
      notifier.decrement();
      
      // Assert
      expect(notifier.count, equals(0));
    });
    
    test('reset should return count to 0', () {
      // Arrange
      notifier.increment();
      notifier.increment();
      expect(notifier.count, equals(2));
      
      // Act
      notifier.reset();
      
      // Assert
      expect(notifier.count, equals(0));
    });
    
    test('should notify listeners on state change', () {
      // Track notification calls
      var notificationCount = 0;
      notifier.addListener(() {
        notificationCount++;
      });
      
      // Act
      notifier.increment();
      
      // Assert
      expect(notificationCount, equals(1));
      
      // Act again
      notifier.increment();
      expect(notificationCount, equals(2));
    });
  });
  
  // Testing Riverpod StateNotifier
  group('CounterState (Riverpod)', () {
    // ProviderContainer allows testing Riverpod providers in isolation
    late ProviderContainer container;
    
    setUp(() {
      container = ProviderContainer();
    });
    
    tearDown(() {
      container.dispose(); // Clean up resources
    });
    
    test('should start at 0', () {
      // Read initial state
      final count = container.read(counterProvider);
      expect(count, equals(0));
    });
    
    test('increment should update state', () {
      // Access the notifier to call methods
      final notifier = container.read(counterProvider.notifier);
      
      // Act
      notifier.increment();
      
      // Read updated state
      final count = container.read(counterProvider);
      expect(count, equals(1));
    });
    
    test('multiple increments accumulate', () {
      final notifier = container.read(counterProvider.notifier);
      
      notifier.increment();
      notifier.increment();
      notifier.increment();
      
      expect(container.read(counterProvider), equals(3));
    });
    
    // Testing with ProviderListener (for side effects)
    test('should trigger listeners on state change', () async {
      final notifier = container.read(counterProvider.notifier);
      final states = <int>[];
      
      // Subscribe to changes
      container.listen(
        counterProvider,
        (previous, next) {
          states.add(next);
        },
      );
      
      // Act
      notifier.increment();
      notifier.increment();
      
      // Allow microtasks to complete (Riverpod is async)
      await Future.delayed(Duration.zero);
      
      // Assert: Should have recorded the new states
      expect(states, equals([1, 2]));
    });
  });
}
```

**Explanation:**

- **ChangeNotifier testing**: For Provider pattern, test the `ChangeNotifier` directly. Verify both the state changes and that `notifyListeners()` is called (via `addListener`).
- **Riverpod `ProviderContainer`**: Riverpod's testing utility that creates an isolated scope for providers. Prevents tests from polluting each other's state.
- **`container.read()`**: Accesses the current value of a provider. Use `.notifier` to get the StateNotifier instance to call methods.
- **`container.listen()`**: Subscribes to state changes, allowing you to collect all state transitions for verification.
- **Async consideration**: Riverpod updates can be asynchronous. Use `await Future.delayed(Duration.zero)` to ensure all microtasks complete before assertions.

---

## **28.4 Asynchronous Testing Patterns**

Testing async code requires handling Futures, timeouts, and ensuring tests don't hang indefinitely.

### **Testing Futures and Timeouts**

```dart
// File: lib/domain/usecases/fetch_data_usecase.dart
class FetchDataUseCase {
  final DataRepository repository;
  
  FetchDataUseCase(this.repository);
  
  Future<Data> execute(String id) async {
    // Simulate network delay
    await Future.delayed(Duration(seconds: 1));
    return await repository.getData(id);
  }
  
  Future<List<Data>> executeMultiple(List<String> ids) async {
    // Fetch all in parallel
    final futures = ids.map((id) => repository.getData(id));
    return await Future.wait(futures);
  }
  
  Future<Data> executeWithTimeout(String id) async {
    return await repository
        .getData(id)
        .timeout(Duration(seconds: 5), onTimeout: () {
      throw TimeoutException('Request timed out');
    });
  }
}

// File: test/domain/usecases/fetch_data_usecase_test.dart
import 'package:flutter_test/flutter_test.dart';
import 'package:mocktail/mocktail.dart';

class MockDataRepository extends Mock implements DataRepository {}

void main() {
  late FetchDataUseCase useCase;
  late MockDataRepository mockRepo;
  
  setUp(() {
    mockRepo = MockDataRepository();
    useCase = FetchDataUseCase(mockRepo);
  });
  
  group('Future testing patterns', () {
    test('should complete with data', () async {
      // Arrange
      final expectedData = Data(id: '1', value: 'test');
      when(() => mockRepo.getData(any()))
          .thenAnswer((_) async => expectedData);
      
      // Act: Call async method
      final result = await useCase.execute('1');
      
      // Assert
      expect(result, equals(expectedData));
    });
    
    test('should throw exception on error', () async {
      // Arrange
      when(() => mockRepo.getData(any()))
          .thenThrow(Exception('Network error'));
      
      // Act & Assert: Use expect with throwsA for async errors
      expect(
        () => useCase.execute('1'),
        throwsA(isA<Exception>()),
      );
    });
    
    test('should complete successfully using completion matcher', () async {
      // Arrange
      when(() => mockRepo.getData(any()))
          .thenAnswer((_) async => Data(id: '1'));
      
      // Act: Don't await, return Future
      final future = useCase.execute('1');
      
      // Assert: completion matcher verifies Future succeeds
      expect(future, completion(isA<Data>()));
      // completion waits for Future and applies matcher to result
    });
    
    test('should throw specific error using throwsA', () async {
      when(() => mockRepo.getData(any()))
          .thenThrow(ArgumentError('Invalid ID'));
      
      expect(
        useCase.execute('1'), // Note: no lambda needed when passing Future
        throwsA(isA<ArgumentError>()),
      );
    });
  });
  
  group('Parallel execution', () {
    test('should fetch multiple items in parallel', () async {
      // Arrange: Setup different return values for different IDs
      when(() => mockRepo.getData('1'))
          .thenAnswer((_) async => Data(id: '1'));
      when(() => mockRepo.getData('2'))
          .thenAnswer((_) async => Data(id: '2'));
      when(() => mockRepo.getData('3'))
          .thenAnswer((_) async => Data(id: '3'));
      
      // Act
      final results = await useCase.executeMultiple(['1', '2', '3']);
      
      // Assert: All returned
      expect(results, hasLength(3));
      expect(results.map((d) => d.id), containsAll(['1', '2', '3']));
    });
  });
  
  group('Timeout handling', () {
    test('should throw TimeoutException when operation exceeds limit', () async {
      // Arrange: Setup delay longer than timeout
      when(() => mockRepo.getData(any()))
          .thenAnswer((_) async {
        await Future.delayed(Duration(seconds: 10)); // Longer than 5s timeout
        return Data(id: '1');
      });
      
      // Act & Assert
      expect(
        () => useCase.executeWithTimeout('1'),
        throwsA(isA<TimeoutException>()),
      );
    });
    
    test('should succeed when operation is within timeout', () async {
      // Arrange: Quick response
      when(() => mockRepo.getData(any()))
          .thenAnswer((_) async => Data(id: '1'));
      
      // Act
      final result = await useCase.executeWithTimeout('1');
      
      // Assert
      expect(result.id, equals('1'));
    });
  });
  
  group('FakeAsync for time manipulation', () {
    test('should respect artificial time advancement', () async {
      // FakeAsync allows controlling time in tests
      await FakeAsync().run((async) async {
        // Arrange
        var completed = false;
        
        // Act: Start delayed operation
        Future.delayed(Duration(seconds: 5), () {
          completed = true;
        });
        
        // Assert: Not completed immediately
        expect(completed, isFalse);
        
        // Advance time by 5 seconds
        async.elapse(Duration(seconds: 5));
        
        // Now it should be complete
        expect(completed, isTrue);
      });
    });
    
    test('should handle periodic timers', () async {
      await FakeAsync().run((async) async {
        var counter = 0;
        
        // Start periodic timer
        Timer.periodic(Duration(seconds: 1), (timer) {
          counter++;
          if (counter >= 3) timer.cancel();
        });
        
        // Advance 3 seconds
        async.elapse(Duration(seconds: 3));
        
        expect(counter, equals(3));
      });
    });
  });
}
```

**Explanation:**

- **`completion` matcher**: Used when you have a Future and want to verify what it completes with. Unlike `await`, this allows the test framework to handle the Future properly.
- **`throwsA` with Future**: When passing a Future directly to `expect()` (not a lambda), the test framework waits for it and checks if it throws.
- **`FakeAsync`**: A testing utility that gives you control over time. `async.elapse()` jumps time forward instantly, allowing you to test time-dependent code without real delays.
- **Periodic timers**: `FakeAsync` is essential for testing `Timer.periodic` or `Stream.periodic` without waiting real time.
- **Microtasks**: Sometimes you need `await Future.delayed(Duration.zero)` to allow the event loop to process pending microtasks before assertions.

---

## **28.5 Stream Testing**

Streams require special handling to test emissions over time, including error events and stream transformations.

### **Testing Stream Emissions**

```dart
// File: lib/domain/blocs/timer_bloc.dart
class TimerBloc {
  final Ticker _ticker;
  StreamSubscription<int>? _tickerSubscription;
  
  // StreamController manages the output stream
  final _stateController = StreamController<TimerState>.broadcast();
  Stream<TimerState> get state => _stateController.stream;
  
  TimerBloc({required Ticker ticker}) : _ticker = ticker;
  
  void start(int duration) {
    _stateController.add(TimerRunInProgress(duration));
    
    // Subscribe to tick stream
    _tickerSubscription?.cancel();
    _tickerSubscription = _ticker.tick(ticks: duration).listen(
      (remaining) {
        if (remaining > 0) {
          _stateController.add(TimerRunInProgress(remaining));
        } else {
          _stateController.add(TimerRunComplete());
        }
      },
      onError: (error) {
        _stateController.add(TimerRunFailure(error.toString()));
      },
    );
  }
  
  void pause() {
    _tickerSubscription?.pause();
    // Get last state and emit paused
    // ... implementation details
  }
  
  void resume() {
    _tickerSubscription?.resume();
  }
  
  void reset() {
    _tickerSubscription?.cancel();
    _stateController.add(TimerInitial());
  }
  
  void dispose() {
    _tickerSubscription?.cancel();
    _stateController.close();
  }
}

class Ticker {
  Stream<int> tick({required int ticks}) {
    return Stream.periodic(
      Duration(seconds: 1),
      (x) => ticks - x - 1,
    ).take(ticks);
  }
}

// States
abstract class TimerState {}
class TimerInitial extends TimerState {}
class TimerRunInProgress extends TimerState {
  final int duration;
  TimerRunInProgress(this.duration);
}
class TimerRunComplete extends TimerState {}
class TimerRunFailure extends TimerState {
  final String error;
  TimerRunFailure(this.error);
}

// File: test/domain/blocs/timer_bloc_test.dart
import 'package:flutter_test/flutter_test.dart';
import 'package:mocktail/mocktail.dart';
import 'package:rxdart/rxdart.dart'; // For advanced stream testing utilities

class MockTicker extends Mock implements Ticker {}

void main() {
  late TimerBloc timerBloc;
  late MockTicker mockTicker;
  
  setUp(() {
    mockTicker = MockTicker();
    timerBloc = TimerBloc(ticker: mockTicker);
  });
  
  tearDown(() {
    timerBloc.dispose();
  });
  
  group('Stream testing', () {
    test('should emit initial state immediately', () async {
      // Expect: First event should be initial state
      await expectLater(
        timerBloc.state,
        emits(isA<TimerInitial>()),
      );
      // emits waits for and verifies a single event
    });
    
    test('should emit sequence of states when started', () async {
      // Arrange: Setup mock tick stream
      when(() => mockTicker.tick(ticks: any(named: 'ticks')))
          .thenAnswer((_) => Stream.fromIterable([2, 1, 0]));
      
      // Act
      timerBloc.start(3);
      
      // Assert: Verify exact sequence of states
      await expectLater(
        timerBloc.state,
        emitsInOrder([
          isA<TimerRunInProgress>().having((s) => s.duration, 'duration', 3),
          isA<TimerRunInProgress>().having((s) => s.duration, 'duration', 2),
          isA<TimerRunInProgress>().having((s) => s.duration, 'duration', 1),
          isA<TimerRunComplete>(),
        ]),
      );
      // emitsInOrder verifies events occur in specific sequence
      // having() checks specific properties of emitted objects
    });
    
    test('should handle stream errors', () async {
      // Arrange: Setup stream that throws error
      when(() => mockTicker.tick(ticks: any(named: 'ticks')))
          .thenAnswer((_) => Stream.error('Connection failed'));
      
      // Act
      timerBloc.start(10);
      
      // Assert: Error state should be emitted
      await expectLater(
        timerBloc.state,
        emits(isA<TimerRunFailure>()),
      );
    });
    
    test('should collect all emissions using emitsDone', () async {
      when(() => mockTicker.tick(ticks: any(named: 'ticks')))
          .thenAnswer((_) => Stream.fromIterable([1, 0]));
      
      timerBloc.start(2);
      
      // emitsDone verifies stream closes after events
      await expectLater(
        timerBloc.state,
        emitsInOrder([
          isA<TimerRunInProgress>(),
          isA<TimerRunInProgress>(),
          isA<TimerRunComplete>(),
          emitsDone, // Verify stream closes
        ]),
      );
    });
  });
  
  group('Stream transformations', () {
    test('should debounce rapid changes', () async {
      // Using rxdart for debounce example
      final stream = Stream.periodic(
        Duration(milliseconds: 100),
        (i) => i,
      ).take(5).debounceTime(Duration(milliseconds: 250));
      // debounceTime emits only after 250ms of silence
      
      await expectLater(
        stream,
        emitsInOrder([4]), // Only last value emitted due to debounce
      );
    });
    
    test('should throttle emissions', () async {
      final stream = Stream.periodic(
        Duration(milliseconds: 100),
        (i) => i,
      ).take(5).throttleTime(Duration(milliseconds: 250));
      // throttleTime emits first, then ignores for 250ms
      
      await expectLater(
        stream,
        emitsInOrder([0, 3]), // First, then one after throttle period
      );
    });
  });
  
  group('StreamSubscription testing', () {
    test('should cancel subscription on reset', () async {
      // Track if subscription was canceled
      var canceled = false;
      final mockStream = Stream<int>.periodic(Duration(seconds: 1));
      final subscription = mockStream.listen((_) {});
      
      // Override cancel to track call
      final originalCancel = subscription.cancel;
      subscription.cancel = () async {
        canceled = true;
        return originalCancel.call();
      };
      
      // Act
      timerBloc.reset();
      
      // Assert: Cleanup occurred
      // (In real test, verify internal state or mock Ticker to track cancel)
    });
  });
  
  // Testing with StreamQueue for interactive stream testing
  group('Interactive stream testing with StreamQueue', () {
    test('should process events interactively', () async {
      final controller = StreamController<int>();
      final queue = StreamQueue(controller.stream);
      // StreamQueue allows requesting events one at a time
      
      // Emit values
      controller.add(1);
      controller.add(2);
      controller.add(3);
      
      // Request and verify one by one
      expect(await queue.next, equals(1));
      expect(await queue.next, equals(2));
      
      // Skip one
      await queue.skip(1);
      
      // Add more and verify
      controller.add(4);
      expect(await queue.next, equals(4));
      
      await controller.close();
      await queue.cancel();
    });
  });
}
```

**Explanation:**

- **`emits` matcher**: Verifies a single event matching the criteria is emitted.
- **`emitsInOrder`**: Critical for testing state management that emits multiple states sequentially. Verifies both the values and the order.
- **`having()`**: Chains property verification. Instead of just checking `isA<TimerRunInProgress>()`, it verifies the `duration` property has the expected value.
- **`emitsDone`**: Verifies the stream closes (completes) after the preceding events. Important for ensuring resources are cleaned up.
- **`StreamQueue`**: From `async` package (used by `rxdart`), allows interactive testing where you request events one at a time rather than asserting the entire sequence upfront.
- **Stream transformations**: Testing operators like `debounce`, `throttle`, `distinct` requires understanding timing. Use `FakeAsync` or controlled `StreamControllers`.

---

## **28.6 Code Coverage and Metrics**

Code coverage measures how much of your code is executed during tests. It's a metric for identifying untested areas, not a goal in itself.

### **Understanding Coverage Metrics**

```dart
// File: lib/domain/utils/string_utils.dart
class StringUtils {
  // This method has multiple branches to demonstrate coverage
  static String formatPhoneNumber(String input) {
    // Remove all non-numeric characters
    final digits = input.replaceAll(RegExp(r'\D'), '');
    
    // Early return for empty
    if (digits.isEmpty) return '';
    
    // Handle different lengths
    if (digits.length == 10) {
      // (123) 456-7890
      return '(${digits.substring(0, 3)}) ${digits.substring(3, 6)}-${digits.substring(6)}';
    } else if (digits.length == 11 && digits.startsWith('1')) {
      // +1 (123) 456-7890
      final withoutCountry = digits.substring(1);
      return '+1 (${withoutCountry.substring(0, 3)}) ${withoutCountry.substring(3, 6)}-${withoutCountry.substring(6)}';
    } else if (digits.length > 10) {
      // International format
      return '+$digits';
    } else {
      // Too short, return as-is
      return digits;
    }
  }
  
  // Untested method (for coverage demonstration)
  static String reverse(String input) {
    return input.split('').reversed.join();
  }
}

// File: test/domain/utils/string_utils_test.dart
// Intentionally incomplete to show coverage gaps

import 'package:flutter_test/flutter_test.dart';

void main() {
  group('formatPhoneNumber', () {
    // This test covers lines: digits cleaning, empty check, and 10-digit branch
    test('should format 10 digit number', () {
      final result = StringUtils.formatPhoneNumber('1234567890');
      expect(result, equals('(123) 456-7890'));
    });
    
    // This covers the 11-digit with country code branch
    test('should format 11 digit number with country code', () {
      final result = StringUtils.formatPhoneNumber('11234567890');
      expect(result, equals('+1 (123) 456-7890'));
    });
    
    // Missing tests:
    // - Empty string input (covers early return)
    // - International number > 11 digits
    // - Short number < 10 digits
    // - reverse() method is completely untested
  });
}
```

**Explanation:**

- **Statement Coverage**: Measures which lines were executed. The example above might have 60% line coverage because `reverse()` and some branches weren't tested.
- **Branch Coverage**: Measures which `if/else` branches were taken. Even with line coverage, you might miss branches (e.g., the `else if (digits.length > 10)` branch).
- **Function Coverage**: Measures which methods were called. `reverse()` has 0% function coverage.
- **Coverage holes**: The comments indicate which scenarios aren't tested. These are gaps that could hide bugs.

### **Generating and Interpreting Coverage Reports**

```bash
# Terminal commands to generate coverage

# 1. Run tests with coverage collection
flutter test --coverage
# Generates coverage/lcov.info file

# 2. Install lcov (if not already installed)
# macOS: brew install lcov
# Linux: sudo apt-get install lcov

# 3. Generate HTML report
genhtml coverage/lcov.info -o coverage/html
# Creates human-readable HTML files showing line-by-line coverage

# 4. Open report
open coverage/html/index.html
```

**Interpreting the Report:**

```dart
// Example of what coverage HTML shows:

// GREEN lines (covered):
final digits = input.replaceAll(RegExp(r'\D'), ''); // Covered
if (digits.isEmpty) return ''; // Condition true covered

// RED lines (not covered):
} else if (digits.length > 10) { // Condition never true in tests
  return '+$digits'; // Never executed
}

// YELLOW branches (partial coverage):
if (digits.length == 10) { // Only true case tested, false branch missed
```

**Coverage Best Practices:**

```dart
// DON'T: Write tests just to hit coverage numbers
test('useless test for coverage', () {
  // This just calls the method without meaningful assertions
  StringUtils.reverse('test');
});

// DO: Test behavior and edge cases
test('reverse should handle palindromes', () {
  expect(StringUtils.reverse('racecar'), equals('racecar'));
});

test('reverse should handle empty string', () {
  expect(StringUtils.reverse(''), equals(''));
});

test('reverse should handle unicode characters', () {
  expect(StringUtils.reverse('hello 🌍'), equals('🌍 olleh'));
});

// Coverage targets by layer:
// - Domain layer: 90-100% (business logic must be fully tested)
// - Data layer: 80-90% (repository coordination, error handling)
// - Presentation layer: 70-80% (UI logic, state transitions)
```

**Explanation:**

- **`flutter test --coverage`**: Generates `lcov.info`, a standard format for coverage data.
- **`genhtml`**: Converts lcov files into HTML websites showing green/red/yellow line highlighting.
- **Meaningful coverage**: 100% coverage with weak assertions is worse than 80% coverage with strong behavioral tests. Focus on testing behaviors, not lines.
- **Layer differences**: Domain logic (calculations, validation) needs near-perfect coverage. UI layer can have lower coverage because some widget code is boilerplate.
- **Exclusions**: Some generated code (freezed, json_serializable) should be excluded from coverage reports using `coverage_excludes` in `dart_test.yaml`.

---

## **Chapter Summary**

In this chapter, we advanced from testing fundamentals to specific patterns for testing business logic and data layers:

### **Key Takeaways:**

1. **Pure Functions**: Test mathematical and validation logic with parametrized tests. Use `closeTo` for floating-point comparisons and test boundary values.

2. **Repository Testing**: Mock external dependencies (network, database). Verify the repository coordinates between sources correctly, handles failures gracefully, and caches appropriately. Use `verify()` and `verifyNever()` to check side effects.

3. **State Management**:
   - **BLoC**: Use `blocTest` from `bloc_test` package to test event-to-state sequences. Verify exact state emissions in order.
   - **Provider**: Test `ChangeNotifier` instances directly, verifying both state changes and listener notifications.
   - **Riverpod**: Use `ProviderContainer` for isolated testing. Test notifiers and state separately.

4. **Asynchronous Testing**: Use `completion` matcher for Futures, `throwsA` for async errors, and `FakeAsync` to control time without real delays. Test timeout behavior and parallel execution.

5. **Stream Testing**: Use `emitsInOrder` for sequence verification, `having()` for property checking, and `StreamQueue` for interactive testing. Remember to test error handlers and stream completion.

6. **Code Coverage**: Generate reports with `flutter test --coverage`. Aim for high coverage in domain layer, but prioritize meaningful behavioral tests over line coverage metrics. Use reports to identify untested branches and edge cases.

### **Next Steps:**

Chapter 29 will cover **Widget Testing**, including:
- Finding widgets using various finder strategies
- Simulating user interactions (tap, scroll, input)
- Testing widget trees and rendering
- Golden tests for visual regression
- Testing different screen sizes and orientations

---

**End of Chapter 28**

---

# **Next Chapter: Chapter 29 - Widget Testing**

Chapter 29 will focus on testing Flutter widgets in isolation, verifying UI behavior, handling user interactions, and preventing visual regressions through comprehensive widget testing strategies.


<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='27. testing_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='29. widget_testing.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
